In [34]:
# find the best config using TERRIER and LLAMA3

import os
import json

exp_path = 'data-out/experiments/PreNormScaleNei/'
l = []
# find al files called log.txt, recursively
for root, dirs, files in os.walk(exp_path):
    for file in files:
        if file.endswith('log.txt'):
            # open file, and for each line test if it contains the string "result for verification run - "
            with open(os.path.join(root, file)) as f:
                for line in f:
                    if 'result for verification run - ' in line:
                        f1 = float(line.split('result for verification run - Strict-F1: ')[1].split(' ')[0])
                        # now, get string between the brackets { and }
                        configs = '{' + line.split('{')[1].split('}')[0] + '}'
                        # replace single quotes with double quotes
                        configs = configs.replace("'", '"')
                        # convert each boolean value to string
                        configs = configs.replace('True', '"True"')
                        configs = configs.replace('False', '"False"')
                        
                        # convert to json
                        config = json.loads(configs)
                        l.append((f1, config))

# sort the list by the first element, which is the f1 score
l.sort(key=lambda x: x[0], reverse=True)

# print the top 10 results
for i, (f1, config) in enumerate(l[:10]):
    print(f'{i}. F1 score: {f1:.4f}, config: {config}')


0. F1 score: 0.7018, config: {'blind_run': 'False', 'split': 'combined', 'preprocess': 'False', 'add_author_name': 'False', 'add_author_bio': 'False', 'out_dir': './data-out/experiments/PreNormScaleNei/0010', 'retriever_k': 5, 'retriever_label': 'TERRIER', 'verifier_label': 'LLAMA', 'normalize_scores': 'False', 'scale': 'True', 'ignore_nei': 'False'}
1. F1 score: 0.6942, config: {'blind_run': 'False', 'split': 'combined', 'preprocess': 'False', 'add_author_name': 'False', 'add_author_bio': 'False', 'out_dir': './data-out/experiments/PreNormScaleNei/0011', 'retriever_k': 5, 'retriever_label': 'TERRIER', 'verifier_label': 'LLAMA', 'normalize_scores': 'False', 'scale': 'True', 'ignore_nei': 'True'}
2. F1 score: 0.6668, config: {'blind_run': 'False', 'split': 'combined', 'preprocess': 'False', 'add_author_name': 'False', 'add_author_bio': 'False', 'out_dir': './data-out/experiments/PreNormScaleNei/0000', 'retriever_k': 5, 'retriever_label': 'TERRIER', 'verifier_label': 'LLAMA', 'normalize_